# 220968280 - Anirudh Swaminathan - DSE A2

## Week 4 - Exercise 3 - KNN algorithm on iris dataset from scratch

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:

iris = pd.read_csv('datasets//iris.csv')
species_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
iris['Species'] = iris['Species'].map(species_map)
iris = iris.drop('Id',axis=1)
iris_df = iris
iris_df['target'] = iris['Species']
iris_df = iris_df.drop('Species', axis=1)
iris_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
iris_df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [4]:
x= iris_df.iloc[:, :-1]
y= iris_df.iloc[:, -1]

In [5]:
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [7]:
x_train, x_test, y_train, y_test= train_test_split(x, y,
                                                   test_size= 0.2,
                                                   shuffle= True, #shuffle the data to avoid bias
                                                   random_state= 0)
x_train= np.asarray(x_train)
y_train= np.asarray(y_train)

x_test= np.asarray(x_test)
y_test= np.asarray(y_test)

In [8]:
scaler= Normalizer().fit(x_train)
normalized_x_train= scaler.transform(x_train) 
normalized_x_test= scaler.transform(x_test) 

In [9]:
def distance_ecu(x_train, x_test_point):
  distances= [] 
  for row in range(len(x_train)):
      current_train_point= x_train[row]
      current_distance= 0
      for col in range(len(current_train_point)):
          current_distance += (current_train_point[col] - x_test_point[col]) **2
      current_distance= np.sqrt(current_distance)
      distances.append(current_distance) 
  distances= pd.DataFrame(data=distances,columns=['dist'])
  return distances


In [10]:
def nearest_neighbors(distance_point, K):
    df_nearest= distance_point.sort_values(by=['dist'], axis=0)
    df_nearest= df_nearest[:K]
    return df_nearest

In [11]:
def voting(df_nearest, y_train):
    counter_vote= Counter(y_train[df_nearest.index])
    y_pred= counter_vote.most_common()[0][0] 
    return y_pred

In [12]:
def KNN_from_scratch(x_train, y_train, x_test, K):
    y_pred=[]
    for x_test_point in x_test:
      distance_point  = distance_ecu(x_train, x_test_point) 
      df_nearest_point= nearest_neighbors(distance_point, K)
      y_pred_point    = voting(df_nearest_point, y_train) 
      y_pred.append(y_pred_point)

    return y_pred  

In [13]:
K=3
y_pred_scratch= KNN_from_scratch(normalized_x_train, y_train, normalized_x_test, K)
print(y_pred_scratch)

[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0]


In [14]:
print(f'accuracy: {accuracy_score(y_test, y_pred_scratch)}')


accuracy: 0.9666666666666667


In [15]:
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
svc = svm.SVC(gamma='scale')
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
kfold = KFold(n_splits=10)
scores = cross_val_score(svc, x, y, cv=kfold)
for i in range(len(scores)):
    print(f"Fold {i+1}: {scores[i]}")
print(f"Average accuracy: {scores.mean()}")

Fold 1: 0.9333333333333333
Fold 2: 0.7333333333333333
Fold 3: 1.0
Fold 4: 1.0
Fold 5: 1.0
Fold 6: 0.9333333333333333
Fold 7: 0.9333333333333333
Fold 8: 1.0
Fold 9: 1.0
Fold 10: 1.0
Average accuracy: 0.9533333333333334


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from tabulate import tabulate
k_values = range(3, 10)
splits = [0.3, 0.25, 0.2]
results = pd.DataFrame(columns=['k', 'Train-Test Split', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
for split in splits:
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=split, random_state=42)
    for k in k_values:
        y_pred = KNN_from_scratch(X_train, y_train, X_test, k)

        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        accuracy = accuracy_score(y_test, y_pred)
        iteration_results = pd.DataFrame({'k': [k], 'Train-Test Split': [split], 'Precision': [precision], 'Recall': [recall], 'F1-score': [f1], 'Accuracy': [accuracy]})
        results = pd.concat([results, iteration_results], ignore_index=True)
results = results.sort_values(by='k')
print(tabulate(results, headers='keys', tablefmt='psql', showindex=False))

C:\Users\s_ani\AppData\Local\Temp\ipykernel_12052\4186865253.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, iteration_results], ignore_index=True)


+-----+--------------------+-------------+----------+------------+------------+
|   k |   Train-Test Split |   Precision |   Recall |   F1-score |   Accuracy |
|-----+--------------------+-------------+----------+------------+------------|
|   3 |               0.3  |    0.934804 | 0.933333 |   0.933456 |   0.933333 |
|   3 |               0.2  |    0.94359  | 0.933333 |   0.932778 |   0.933333 |
|   3 |               0.25 |    0.922368 | 0.921053 |   0.920835 |   0.921053 |
|   4 |               0.3  |    0.934804 | 0.933333 |   0.933456 |   0.933333 |
|   4 |               0.2  |    0.94359  | 0.933333 |   0.932778 |   0.933333 |
|   4 |               0.25 |    0.922368 | 0.921053 |   0.920835 |   0.921053 |
|   5 |               0.3  |    0.960784 | 0.955556 |   0.955637 |   0.955556 |
|   5 |               0.2  |    0.94359  | 0.933333 |   0.932778 |   0.933333 |
|   5 |               0.25 |    0.953947 | 0.947368 |   0.94693  |   0.947368 |
|   6 |               0.2  |    0.94359 